# Quick Start Tutorial: Feature Management

## Learning Objectives

In this tutorial you will learn:
1. How to view the lineage of a feature
2. How to manage the readiness of a feature
3. How to be informed of the readiness of a feature list
4. How to manage the status of a feature list
5. How FeatureByte deployment guardrails work
6. How to check the feature job status
7. How to manage versioning
8. How to disable deployment

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn about catalogs
* activate a pre-built catalog

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartFeatureManagement)

## Manage Feature Readiness

Learning Objectives

In this section you will learn:
* how to change the readiness of a feature
* the meaning of each readiness value

### Feature readiness

To help differentiate features that are in the prototype stage and features that are ready for production, a feature version can have one of four readiness levels:

PRODUCTION_READY: ready for deployment in production environments.<br>
PUBLIC_DRAFT: shared for feedback purposes.<br>
DRAFT: in the prototype stage.<br>
DEPRECATED`: not advised for use in either training or prediction.

In [3]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

### Example: Set features to production ready

In [4]:
# change the state features to be production ready
for feature_name in catalog.list_features().name:
    feature = catalog.get_feature(feature_name)

    # does the feature name contain the word "state"?
    if "State" in feature.name:
        feature.update_readiness("PRODUCTION_READY")

In [5]:
# list the features in the catalog - note the readiness of each
catalog.list_features()

## List Unsaved Features

Features that have not been saved will not persist once you close your Notebook. It is helpful to check that you have saved each feature that you wished to keep.

In [6]:
# create a feature without saving it
grocery_items_table = catalog.get_table("INVOICEITEMS")
grocery_items_view = grocery_items_table.get_view()
invoice_unique_product_ids = grocery_items_view.groupby(
    "GroceryInvoiceGuid", category="GroceryProductGuid"
).aggregate(None, method=fb.AggFunc.COUNT, feature_name="InvoiceUniqueProductIds")
invoice_unique_product_count = invoice_unique_product_ids.cd.unique_count()

In [7]:
# list unsaved features
fb.list_unsaved_features()

# note that the feature we just created hasn't been named, so cannot be saved

## Manage Feature List Status

Learning Objectives

In this section you will learn:
* how to change the status of a feature list
* the meaning of each status value
* how to deploy a feature list

### Feature list status

Feature lists can be assigned one of five status levels to differentiate between experimental feature lists and those suitable for deployment or already deployed.

- DEPLOYED: Assigned to feature list with at least one deployed version.
- TEMPLATE: For feature lists as reference templates or safe starting points.
- PUBLIC_DRAFT: For feature lists shared for feedback purposes.
- DRAFT: For feature lists in the prototype stage.
- DEPRECATED: For outdated or unnecessary feature lists.

In [8]:
# list the feature lists in the catalog - note the status of each
# Note the readiness fraction which represents the proportion of features that are production ready
# Note the online fraction which represents the proportion of features that are being used in production
catalog.list_feature_lists()

### Example: Make a feature list public

When a feature list is reviewed and ready for sharing with other users, change its status to PUBLIC_DRAFT

In [9]:
# get the state feature list
state_feature_list = catalog.get_feature_list("StateFeatureList")

# update the status to public draft
state_feature_list.update_status("PUBLIC_DRAFT")

In [10]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

### Example: Deploy a feature list

Deploying a feature list changes its status to published

In [11]:
# deploy the state feature list
deployment = state_feature_list.deploy(make_production_ready=True)
deployment.enable()

In [12]:
# list the feature lists in the catalog - note the status of each
catalog.list_feature_lists()

## Versioning

Learning Objectives

In this section you will learn:
* about feature versions and feature list versions
* how to change the table cleaning operations for a feature
* how to change the feature job settings for a feature
* how to manage the default versions for a feature or a feature list
* how to create new versions of features and feature lists

### Concept: Feature version
A Feature Version enables the reuse of a Feature with varying feature job settings or distinct cleaning operations.

If the availability or freshness of the data source change, new versions of the feature can be generated with a new feature job setting. On the other hand, if changes occur in the data quality of the data sources, new versions of the feature can be created with new cleaning operations that address the new quality issues.

To ensure the seamless inference of Machine Learning tasks that depend on the feature, old versions of the feature can still be served without any disruption.

### Example: Get table cleaning operations for a feature

In [13]:
# get the InvoiceDiscountAmount feature
invoice_discount_amount = catalog.get_feature("InvoiceDiscountAmount")

# list the feature versions
display(invoice_discount_amount.list_versions())

In [14]:
# No cleaning has been operated on this feature
invoice_discount_amount.info()["table_cleaning_operation"]

### Example: Updating table cleaning operations for a feature

In [15]:
# update the data cleaning operations in the InvoiceDiscountAmount feature
new_version = invoice_discount_amount.create_new_version(
    table_cleaning_operations=[
        fb.TableCleaningOperation(
            table_name="INVOICEITEMS",
            column_cleaning_operations=[
                fb.ColumnCleaningOperation(
                    column_name="Discount",
                    cleaning_operations=[
                        fb.MissingValueImputation(imputed_value=0.0),
                        fb.ValueBeyondEndpointImputation(
                            type="less_than", end_point=0, imputed_value=None
                        ),
                    ],
                )
            ],
        )
    ]
)

In [16]:
# list the feature versions
feature_versions = invoice_discount_amount.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# display only the InvoiceUniqueProductGroups feature - note the new version that has been created
display(feature_versions)

In [17]:
# Check new cleaning info
new_version.info()["table_cleaning_operation"]["this"]

In [18]:
# Check feature definition file
new_version.definition

### Concept: Default feature version
    
The default version of a feature streamlines the process of reusing features by providing the most appropriate version. Additionally, it simplifies the creation of new versions of feature lists.

By default, the feature's version with the highest level of readiness is considered, unless the user overrides this selection. In cases where multiple versions share the highest level of readiness, the most recent version is automatically chosen as the default.

When a feature is accessed without specifying a version ID but only by its name, the default version is automatically retrieved.

### Example: The version we just created should be the default as no other version has a higher readiness and it is the latest version.

In [19]:
new_version.is_default

### Example: Get the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [20]:
# get the CustomerInventory_28d feature
customer_inventory_28d_feature = catalog.get_feature("CustomerInventory_28d")

# list the feature versions
display(customer_inventory_28d_feature.list_versions())

In [21]:
# Get the feature job settings for CustomerInventory_28d
customer_inventory_28d_feature.info()["table_feature_job_setting"]["this"]

Note that the table name here is the event table associated with the item table as the timestamp originates from this table.

In [22]:
# show the feature job settings for the grocery invoice table
grocery_invoice_table = catalog.get_table("GROCERYINVOICE")
grocery_invoice_table.default_feature_job_setting

### Example: Change the feature job settings for a feature

Note that changing feature job settings will only affect time-aware features e.g. features created using aggregate_over. It will not affect features based upon simple aggregation.

In [23]:
# update the data cleaning operations for the InvoiceDiscountAmount feature to be more conservative
new_version = customer_inventory_28d_feature.create_new_version(
    table_feature_job_settings=[
        fb.TableFeatureJobSetting(
            table_name="GROCERYINVOICE",
            feature_job_setting=fb.FeatureJobSetting(
                blind_spot="160s",
                frequency="60m",
                time_modulo_frequency="90s",
            ),
        ),
    ]
)

In [24]:
# list the feature versions
feature_versions = customer_inventory_28d_feature.list_versions()

# sort by created_at ascending
feature_versions.sort_values(by="created_at", ascending=True, inplace=True)

# note that the new version is a draft, and that the old version remains production ready
display(feature_versions)

In [25]:
# Get the feature job settings for the new version
new_version.info()["table_feature_job_setting"]["this"]

In [26]:
# Check feature definition file
new_version.definition

### Example: Change default feature version mode

The new version of CustomerInventory_28d is not the default as this version is a draft and the prior version is production ready.

In [27]:
new_version.is_default

The default can be changed only if the default version mode is set as manual.

In [28]:
# guardrail if default version mode is not MANUAL
# the new version cannot be set as the default
try:
    new_version.as_default_version()
except Exception as ex:
    print(ex)

In [29]:
# downgrade current feature readiness to public draft first
customer_inventory_28d_feature.update_readiness("PUBLIC_DRAFT")

# upgrade new version readiness to public draft, new version becomes default
new_version.update_readiness("PUBLIC_DRAFT")
print(new_version.is_default, customer_inventory_28d_feature.is_default)

In [ ]:
# change mode to manual and set the original version as default
customer_inventory_28d_feature.update_default_version_mode("MANUAL")
customer_inventory_28d_feature.as_default_version()
print(customer_inventory_28d_feature.is_default)

In [30]:
# change new version as default
new_version.as_default_version()
print(new_version.is_default)

In [ ]:
# upgrade new version readiness to production ready
new_version.update_readiness("PRODUCTION_READY", ignore_guardrails=True)

### Example: Cannot have more than one production ready version of a feature

In [32]:
# change the readiness of the original version of CustomerInventory_28d to production ready
try:
    customer_inventory_28d_feature.update_readiness("PRODUCTION_READY")
except Exception as ex:
    print("Error changing the readiness of the new version to production ready")
    print(ex)

### Example: Create version of a Feature List

The Feature List Version allows the use of the latest version of each feature. Upon creation of a new feature list version, the latest default versions of features are employed, unless particular feature versions are specified.

In [33]:
# the current default of the feature list has all feature versions production ready
customer_feature_list = catalog.get_feature_list("CustomerFeatureList")
customer_feature_list.list_features()

In [34]:
# create new version with the new default of CustomerInventory_28d
new_feature_list_version = customer_feature_list.create_new_version()

In [35]:
new_feature_list_version.list_features()

In [36]:
# check readiness
new_feature_list_version.info()["production_ready_fraction"]

### Default version of a Feature List

The default version of a feature list is the version with the highest fraction of production ready features.

The new version is not the default as its production_ready_fraction is lower than the prior version for CustomerInventory_28d.

In [37]:
# the new version is not the default as it is production_ready_fraction is lower than the current default
new_feature_list_version.is_default

## Deleting Drafts

While prototyping, you may create and experiment with many features and feature lists. To avoid feature explosions you should do regular cleanups of unused features and feature lists. Note that you cannot delete features or feature lists that have ever been deployed.

### Example: Deleting a feature

In [38]:
# list all of the features in this catalog
display(catalog.list_features())

# note the feature called "unused experimental feature" - this is a feature that is not being used in production, and was rejected as unhelpful

In [39]:
# get the feature
feature_to_delete = catalog.get_feature("unused experimental feature")

# delete the feature
feature_to_delete.delete()

In [40]:
# list all the features and note that the deleted feature no longer appears
display(catalog.list_features())

### Example: Deleting a feature list

In [41]:
# list all of the feature lists in the catalog
display(catalog.list_feature_lists())

# note the feature list called "very short feature list" - this is a feature list that is not being used in production, and was rejected as unhelpful

In [42]:
# get the feature list
feature_list_to_delete = catalog.get_feature_list("very short feature list")

# delete the feature
feature_list_to_delete.delete()

In [43]:
# list all the feature lists and note that the deleted feature list no longer appears
display(catalog.list_feature_lists())

## Deployment Guardrails

FeatureByte has guardrails for deployment, and will prevent you from deploying a feature list that has features that are not production ready.

Learning Objectives

In this section you will learn:
* how to check the readiness of a feature list
* how to deploy a feature list

### Example: Check readiness of a feature list

The Feature List Readiness metric provides information to users about the readiness status of a Feature List. This metric represents the percentage of features that are production ready within the given feature list.

In [44]:
# get the invoice feature list
invoice_feature_list = catalog.get_feature_list("InvoiceFeatureList")

# check feature list is ready to be deployed
invoice_feature_list.info()["production_ready_fraction"]

In [45]:
invoice_feature_list.list_features()

### Example: Deploy a feature list when production readiness is not 100%

In [46]:
# deploy the invoice feature list
try:
    bad_deployment = invoice_feature_list.deploy()
    bad_deployment.enable()
except Exception as ex:
    print("Error deploying the invoice feature list")
    print(ex)

In [47]:
# show the feature lists - note that the invoice feature list has not been deployed
catalog.list_feature_lists()

## Disable Deployment

Learning Objectives

In this section you will learn:
* how to disable deployments

### Example: Disabling a deployment

In [48]:
# helper function to disable deployment for a specific feature list
def disable_deployment(feature_list_name):
    # list deployments
    deployments = catalog.list_deployments()

    # just the ones matching this feature list name
    deployments = deployments.loc[deployments.feature_list_name == feature_list_name]

    # disable
    for id in deployments.id:
        deployment = catalog.get_deployment_by_id(id)
        deployment.disable()

In [49]:
# disable the deployments
disable_deployment("CustomerFeatureList")
disable_deployment("StateFeatureList")

In [50]:
# show the feature lists status
catalog.list_feature_lists()

## Next Steps

Now that you've completed the quick-start feature management tutorial, you can put your knowledge into practice or learn more:<br>
1) Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" catalogs
2) Learn more about feature engineering via the "Deep Dive Feature Engineering" tutorial
3) Learn about data modeling via the "Deep Dive Data Modeling" tutorial